In [2]:
import pandas as pd
import xgboost as xgb
import joblib
import numpy as np
from sklearn.preprocessing import LabelEncoder

print("🔧 STARTING AUTO-FIX & ALIGNMENT...")

# 1. Load the Data
try:
    df = pd.read_csv('scaled_simulation.csv')
    print(f"   - Loaded data: {df.shape}")
except FileNotFoundError:
    print("❌ Error: 'scaled_simulation.csv' not found.")
    exit()

# 2. Prepare Data
# We assume the last column is the label
target_col = 'Label_Encoded'
if target_col not in df.columns:
    target_col = df.columns[-1]

y_raw = df[target_col]
X = df.drop(target_col, axis=1)
features = X.columns.tolist()

# 3. FIX: Renumber Non-Consecutive Labels
# If labels are [0, 1, 2, 6], this makes them [0, 1, 2, 3] so XGBoost is happy
le = LabelEncoder()
y_fixed = le.fit_transform(y_raw)

print(f"   - Original Labels: {np.unique(y_raw)}")
print(f"   - Fixed Labels:    {np.unique(y_fixed)}")

# 4. Train Model on Fixed Labels
print("   - Retraining model on fixed labels...")
model = xgb.XGBClassifier(n_estimators=50, use_label_encoder=False, eval_metric='logloss')
model.fit(X, y_fixed)

# 5. Create the "Translator" Dictionary
# This maps the NEW numbers back to the REAL Threat Names
# Global truth based on your dataset
GLOBAL_THREAT_MAP = {
    0: "BENIGN",
    1: "Bot",
    2: "BruteForce",
    3: "DoS",
    4: "Infiltration",
    5: "PortScan",
    6: "WebAttack"
}

# We map: Model Prediction (0,1,2,3) -> Original Code (0,1,2,6) -> Name ("WebAttack")
final_label_map = {}
for model_code, original_code in enumerate(le.classes_):
    threat_name = GLOBAL_THREAT_MAP.get(original_code, f"Type-{original_code}")
    final_label_map[model_code] = threat_name

print(f"   - Generated Label Map: {final_label_map}")

# 6. Save Everything
joblib.dump(model, 'ids_model.pkl')
joblib.dump(features, 'selected_features.pkl')
joblib.dump(final_label_map, 'label_map.pkl')

print("\n✅ SUCCESS: System repaired.")
print("👉 Restart your Streamlit App and upload 'scaled_simulation.csv'.")

🔧 STARTING AUTO-FIX & ALIGNMENT...
   - Loaded data: (200, 30)
   - Original Labels: [0 1 2 3 6]
   - Fixed Labels:    [0 1 2 3 4]
   - Retraining model on fixed labels...
   - Generated Label Map: {0: 'BENIGN', 1: 'Bot', 2: 'BruteForce', 3: 'DoS', 4: 'WebAttack'}

✅ SUCCESS: System repaired.
👉 Restart your Streamlit App and upload 'scaled_simulation.csv'.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [20:04:40] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
